# Foundation Model Prompt Engineering: Text2Text Generation


For this lab, we are going to be using a pre-hosted AI21Labs Jurrasic Mid model for the various prompting scenarios. The URL below points to an API pointing to a sagemaker model endpoint running Jurrasic Mid. 

If you wish to try the prompts against your own endpoint simply deploy an instance of Jurrasic model variant..

Happy Prompting! 

In [ ]:
import logging
import json
import time
import boto3

import matplotlib.pyplot as plt
import numpy as np
import requests
from tqdm.contrib.concurrent import thread_map

region = boto3.Session().region_name

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())
#url = "https://yyosweaa3e.execute-api.us-east-1.amazonaws.com/LATEST/HF"
url = "https://nm3yyjazj1.execute-api.us-east-1.amazonaws.com/Prod/invoke"

In [ ]:
def query_endpoint_with_json_payload(url, payload):
    response = requests.post(
        url,
        json=payload,
    )
    #print(payload)
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = query_response.json()
    #print(query_response)
    generated_text = model_predictions['message']
    return generated_text

### Advanced features

***
This model also supports many advanced parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **num_return_sequences:** Number of output sequences returned. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **seed:** Fix the randomized state for reproducibility. If specified, it must be an integer.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments

***

## N-shot Learning via In-context Learning

Let's put in some example input text. You can put any text containing the task, the model returns the output of the accomplished task.

Payload must be a json

### A. Zero-shot Learning

### Text summarisation - generate text and summarise text, Use cases: creating articles, writing product descriptions, etc

#### Example 1 - Generate text

In [ ]:
instruction = 'Explain Australia’s foreign policy'

prompt = instruction

print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
#llm_response = response.json()["message"]
#print(llm_response)
generated_text = parse_response_multiple_texts(response)
print(f'Generated text: \n{generated_text}')

#### Example 2 - Text summarisation

In [ ]:
context = generated_text
print(f'Context: \n{context}')

In [ ]:
instruction = "Explain the above in one sentence"

prompt = f'{context}\n{instruction}:'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Generated text: \n{generated_text}')

#### Sentiment Analysis/Classification

#### Example 3: generate text, product description

In [ ]:
instruction = "Generate a concise product description for the product: laptop."

prompt = f'{instruction}:'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Generated text: \n{generated_text}')

#### Example 4: Lets control the output of product description, indicating the sections you would like the model to generate

In [ ]:
instruction = "Generate a concise product description for the product: laptop."
output_indicator = "Use the following format: Hook, Solution, Features and Benefits, Call to Action."

prompt = f'{instruction}\n{output_indicator}'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Generated text: \n{generated_text}')

### 2. Information extraction: Extract keywords, location, people, sentiment. Use cases: SEO, metadata extraction

The example is an excerpt from the article : https://www.sbs.com.au/news/article/anthony-albanese-to-hold-talks-with-joe-biden-and-rishi-sunak-on-aukus-defence-pact/0ar8pshdb


#### Extract location from the article

In [ ]:
article_context = "US President Joe Biden will host leaders of Australia and the United Kingdom \
in San Diego next week to chart a way forward for the provision of nuclear-powered submarines and \
other high-tech weaponry to Australia, sources familiar with the plans said. \
The spokesperson for UK Prime Minister Rishi Sunak said he will visit the United States on Monday \
to meet Mr Biden and Prime Minister Anthony Albanese for talks on the AUKUS defence agreement."


In [ ]:
instruction = "List the locations mentioned in the paragraph above:"

prompt = f'{article_context}\n{instruction}'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Locations: \n{generated_text}')

#### Extract people from the article

In [ ]:
instruction = "List the people mentioned in the paragraph above:"

prompt = f'{article_context}\n{instruction}'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'People mentined in the article: \n{generated_text}')

#### Extract keywords from the article

In [ ]:
instruction = "Extract keywords from the paragraph above:"

prompt = f'{article_context}\n{instruction}'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Keywords in the article: \n{generated_text}')

#### Extract sentiment from the article

In [ ]:
article_context = 'Prime Minister Anthony Albanese is expected to announce Australias pathway to getting nuclear-powered subs in San Diego on Tuesday (AEDT) alongside Mr Biden and Mr Sunak. He will then hold bilateral meetings with both. \
"This is a joint arrangement between Australia, the United States, and the United Kingdom," he told reporters on Thursday during a visit to India. \
"We are great friends. We have over a century of standing side by side during peacetime and during conflict.'


In [ ]:
instruction = "What is the sentiment in the paragraph above? Reply as Neutral, Positive, or Negative only"

prompt = f'{article_context}\n\n{instruction}'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Sentiment of the article: \n{generated_text}')

#### 3. Question answering from the article

In [ ]:
article_context = "Novak Djokovic reaches record 23 grand slam titles after French Open final win\n\nSerb beats Casper Ruud 7-6 (1), 6-3, 7-5 at Roland Garros\nDjokovic now has one more grand slam title than Rafael Nadal\n\nSince his emergence at the top level of professional tennis 18 years ago, Novak Djokovic has used the dizzying bar set by Roger Federer and Rafael Nadal before him as inspiration to push himself to his limits, never doubting that he would one day rise above it.\n\nWhat once seemed impossible eventually became inevitable. On Sunday, Djokovic finally surpassed his great rivals in the most significant category of all as he defeated Casper Ruud 7-6 (1), 6-3, 7-5 to clinch his 23rd grand slam title, breaking his tie of 22 with Nadal.\n"

In [ ]:
question = "Whom did Djokovic defeat and what was the score?"

In [ ]:
prompt = f'{article_context}\n{question}'
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Answer: \n{generated_text}')

#### 4. K-Shot prompt example

#### Example - Lets check the model output without providing the examples of what we expect the model to respond back

In [ ]:
prompt = f"""
Review: "I really had fun watching this movie"
This review is:

"""
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Answer: \n{generated_text}')

#### Example 2: 
As you can see above, we might expect the model to respond back with positive, nuetral or negative.
But it does not and responds back with a more generic outputlets see how we can provide some examples to the model on what we expect 

In [ ]:
prompt = f"""

Review: "I loved this movie!"
This review is positive.

Review: "I am not sure, I think the movie was fine."
The review is neutral.

Review: "This movie was a waste of time and money."
This review is negative.

Review: "I really had fun watching this movie"
This review is:

"""
print(f'Prompt: {prompt}')

In [ ]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Answer: \n{generated_text}')

## Challenge 1: Create a personalised email for a customer 

Create an email for a customer based on recommended movies they can watch as well as customer profile information. Let us for a moment pretend that we have made an API call to our CRM and know about the customer profile. The sample below shows a json payload of customer profile. 


In [ ]:
customer_data = {
    "name":"Peter"
    "age":"30"
    "genre":["Fantasy", "Sci-Fi","Action"]
}

Now let us pretend that we have a list of movie recommendations returned by Amazon Personalize recomemmender system. If you wish to find out more about Amazon Personalize on this link - https://aws.amazon.com/personalize/. 

In [ ]:
recommendations = {
    "movies":
        [
            {
                "title":"Lord of the rings",
                "synopsis":"A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron."
            },
            {
                "title":"India Jones and the Raiders of the Lost Ark",
                "synopsis":"Epic tale in which an intrepid archaeologist tries to beat a band of Nazis to a unique religious relic which is central to their plans for world domination. Battling against a snake phobia and a vengeful ex-girlfriend, Indiana Jones is in constant peril, making hair's-breadth escapes at every turn in this celebration of the innocent adventure movies of an earlier era."
            },
            {
                "title":"Star Wars Episode IV - A New Hope",
                "synopsis":"The Imperial Forces -- under orders from cruel Darth Vader (David Prowse) -- hold Princess Leia (Carrie Fisher) hostage, in their efforts to quell the rebellion against the Galactic Empire. Luke Skywalker (Mark Hamill) and Han Solo (Harrison Ford), captain of the Millennium Falcon, work together with the companionable droid duo R2-D2 (Kenny Baker) and C-3PO (Anthony Daniels) to rescue the beautiful princess, help the Rebel Alliance, and restore freedom and justice to the Galaxy."
            }
        ]
}

Below we are providing a sample prompt that you can use to create a prompt using the information to generate a personalised, and themed email copy created for the customer. The great thing about this approach is the ability to tailor emails for the individual so that not one customer receives the same email.

The final prompt should look like the following. If you are struggling with the prompt or wish to avoid writing the code then simply copy and paste this prompt in the cell below and continue. - 

--------

Peter is has an active subscription of ACME streaming service. He generaly watches Fantasy, Sci-fi and Action movies and titles. 

Write a detailed professional and perosnalized marketing email from ACME customer experience team addressing Peter recommendings the titles outlined below. Only recommend movie titles from below. Tailor the email to someone who enjoyes science fiction, action and fantasy genre. Mention in the email that Nagib can browse other titles from our vast library of on demand content. 

Movie Recommendations: 
 
Title: Lord of the rings
Syponsis:  A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.

Title: India Jones and the Raiders of the Lost Ark
Sysponsis: Epic tale in which an intrepid archaeologist tries to beat a band of Nazis to a unique religious relic which is central to their plans for world domination. Battling against a snake phobia and a vengeful ex-girlfriend, Indiana Jones is in constant peril, making hair's-breadth escapes at every turn in this celebration of the innocent adventure movies of an earlier era.

Title: Star Wars Episode IV - A New Hope
Syponsis: The Imperial Forces -- under orders from cruel Darth Vader (David Prowse) -- hold Princess Leia (Carrie Fisher) hostage, in their efforts to quell the rebellion against the Galactic Empire. Luke Skywalker (Mark Hamill) and Han Solo (Harrison Ford), captain of the Millennium Falcon, work together with the companionable droid duo R2-D2 (Kenny Baker) and C-3PO (Anthony Daniels) to rescue the beautiful princess, help the Rebel Alliance, and restore freedom and justice to the Galaxy.

---------


In [ ]:
# TODO - complete the prompt to include the users detail as well as the recommendations. Do not forget to ask the LLM to write you an email copy. 
prompt = ""


Now that we have a prompt created with the necessary context + data, let us call the LLM to get the email copy. 

In [ ]:
payload = payload = {"prompt":prompt, "max_token":200, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
print(f'Answer: \n{generated_text}')

## Challenge 2: Text 2 Image Challenge using Stable Diffusion 

We can use stable diffusion to create stock images that can accompany freshly authored articles without having to pay for licensing and copyright. Since all images are created using the generative AI model we can use for various types of publications. 

Let us pretend, there is an article published on "Green cities". If you are after a more realistic scenario please view this article - 
https://www.smh.com.au/traveller/reviews-and-advice/green-cities-10-major-cities-embracing-green-space-sustainability-and-renewable-energy-20221118-h27yle.html 

Our objective us to create a green city image that we can use as concept image for the article. Here are some sample prompts that we have used in our tests - 

Prompt: 
    beautiful and ultra realistic Paris city landscape dotted with rooftop gardens, tree-sprouting buildings and landmarks, Arc de triomphe, eiffel tower, Greentech, green cities walkable cities, sprawling plant life, an abundance of sunshine, Art Nouveau, Green architecture, towering vertical forests, Renewable energies, solar power, rainwater harvesting, self-sustainable community gardening, decentralized technologies, beautiful clear sunny day. - photo realistic, high quality, 4k, ultra realistic

Negative Prompt: 
    monochrome, grayscale, cartoon, oil painting, painting, animated
    


Use the above as inspirations to create your own style and imagery. Use the stable diffision endpoint that you deployed earlier. 